# Dark current: the ideal case

In [ ]:
import numpy as np
from scipy import stats

%matplotlib inline
from matplotlib import pyplot as plt

from image_sim import dark_current, read_noise


## A dark frame measures dark current

Recall that *dark current* refers to counts (electrons) generated in a pixel because an electron in the pixel happens to have enough energy to "break free" and register as a count. The distribution of electron thermal energies in  pixel follows a [Maxwell-Boltzmann distribution](https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution) in which most electrons have energy around $kT$, where $T$ is the temperature of the sensor and $k$ is the Boltzmann constant. There is a distribution of energies, though, and occasionally an electron will be high enough energy to jump to the conducting band in the chip, registering the same as an electron excited by a photon. Since the Maxwell-Boltzmann distribution depends on temperature the rate at which dark current appears in a pixel is also expected to depend on temperature. 

A *dark frame* (also called a *dark image*) is an image taken with your camera with the shutter closed. It is the sum of the bias level of your camera, the readout noise, and the dark current.

You measure the dark current in your camera by taking dark frames.

## Dark current theory

The expected signal in a dark frame exposure of time $t$ is proportional to $t$. If we call the dark electrons in an exposure $d_e(t)$ and the dark current $d_c(T)$, where $T$ is the temperature, then 

$$
d_e(t) = d_c(T) t.
$$

For liquid-cooled cameras, particularly ones cooled bu liquid nitrogen, the operating temperature doesn't change. For thermo-electrically cooled cameras one is able to set the desired operating temperature. As a result, you should be able to ignore the temperature dependence of the dark current.

The thermo-electric coolers can usually cool by some fixed amount below the ambient temperature. Though in principle one could choose to always cool by the same fixed amount, like $50^\circ$C below the ambient temperature, there is an advantage to always running your camera at the same temperature: dark frames taken on one date are potentially useful on another date. If the operating temperature varies then you need to make sure to take dark frames every time you observe unless you carefully characterize the temperature dependence of your dark current. 

It will turn out that for practical reasons -- not all pixels in your camera have the same dark current -- it is usually best to take dark frames every time you observe anyway.

### Illustration with dark current only, no read noise

For the purposes of illustrating some of the properties of dark current and dark frames we'll generated some simulated images in which the counts are due to dark current alone. We'll use these values:

+ Dark current is $d_c(T) = 0.1 e^-$/pixel/sec
+ Gain is $g = 1.5 e^-$/ADU
+ Read noise is 0 $e^-$

In [ ]:
dark_rate = 0.1
gain = 1.5
read_noise_electrons = 0

#### Dark current is a random process

The dark counts in a dark frame are counts and so they follow a Poisson distribution. The plot below shows the dark current in a number of randomly chosen pixels in 20 different simulated images each with exposure time 100 sec. Note that the counts vary from image to image but that the average is very close to the expected value.

The expected value of the dark counts for this image are $d_e(t)/g = 6.67~$counts.

In [ ]:
exposure = 100

n_images = 20
n_pixels = 10
image_size = 500

pixels = np.random.randint(50, high=190, size=n_pixels)
pixel_values = np.zeros(n_images)
pixel_averages = np.zeros(n_images)
base_image = np.zeros([image_size, image_size])

plt.figure(figsize=(20, 10))
for pixel in pixels:
    for n in range(n_images):
        a_dark = dark_current(base_image, dark_rate, exposure, gain=gain, hot_pixels=False)
        pixel_values[n] = a_dark[pixel, pixel]

    plt.plot(pixel_values, label='pixel [{0}, {0}]'.format(pixel), alpha=0.5)
    pixel_averages += pixel_values

plt.plot(pixel_averages / n_pixels, 
         linewidth=3,
         label='Average over {} pixels'.format(n_pixels))
# plt.xlim(0, n_images - 1)
plt.hlines(dark_rate * exposure / gain, *plt.xlim(), 
           linewidth=3, 
           label="Expected counts")
plt.xlabel('Image number')
plt.ylabel('Counts due to dark current')

plt.legend()
plt.grid()

#### The distribution of dark counts follows a Poisson distribution

The distribution below shows a normalized histogram of number of pixels as a function of dark counts in each pixel for one of the simulated dark frames. Overlaid on the histogram is a Poisson distribution with a mean of $d_e(t_{exp}) = d_C(T) * t_{exp} / g$, where $t_{exp}$ is the exposure time.

In [ ]:
plt.figure(figsize=(20, 10))


h = plt.hist(a_dark.flatten(), bins=20, align='mid', density=True, label="Histogram of dark current counts");
bins = h[1]
pois = stats.poisson(dark_rate * exposure / gain)
pois_x = np.arange(0, 20, 1)

plt.plot(pois_x, pois.pmf(pois_x), 
         label="Poisson dsitribution, mean of {:5.2f} counts".format(dark_rate * exposure / gain))  

plt.xlabel("Dark counts in {} exposure".format(exposure))
plt.ylabel("Number of pixels (area normalized to 1)")
plt.legend()
plt.grid()

### Illustration with dark current *and* read noise

Now let's run through the same couple of plots with a non-zero read noise. For the sake of illustration, we'll look at two cases:

1. Moderate read noise of 10 $e^-$ per read, typical of a low-end research-grade CCD
2. Low read noise of 1 $e^-$ per read

In both cases we'll continue with the parameters above to generate our frames:

+ Dark current is $d_c(T) = 0.1 e^-$/pixel/sec
+ Gain is $g = 1.5 e^-$/ADU
+ Exposure time 100 sec

With those choices the expected dark count is 6.67 count, which is 10 $e^-$. That is, not coincidentally, one of the values for read noise that was chosen.

### Read noise is about the same as the expected dark current

In this first case, the read noise and the dark current are both 10$e^-$.

In [ ]:
high_read_noise = 10

In [ ]:
pixels = np.random.randint(50, high=190, size=n_pixels)
pixel_values = np.zeros(n_images)
pixel_averages = np.zeros(n_images)
base_image = np.zeros([image_size, image_size])
darks = np.zeros([n_images, image_size, image_size])

plt.figure(figsize=(20, 10))
for n in range(n_images):
    darks[n] = dark_current(base_image, dark_rate, exposure, gain=gain, hot_pixels=False)
    darks[n] = darks[n] + read_noise(base_image, high_read_noise, gain=gain)
for pixel in pixels:
    for n in range(n_images):
        pixel_values[n] = darks[n, pixel, pixel]
    plt.plot(pixel_values, label='pixel [{0}, {0}]'.format(pixel), alpha=0.5)
    pixel_averages += pixel_values

image_average = darks.mean(axis=0)

plt.plot(pixel_averages / n_pixels, 
         linewidth=3,
         label='Average over {} pixels'.format(n_pixels))
# plt.xlim(0, n_images - 1)
plt.hlines(dark_rate * exposure / gain, *plt.xlim(), 
           linewidth=3, 
           label="Expected counts")
plt.xlabel('Image number')
plt.ylabel('Counts due to dark current')

plt.legend()
plt.grid()

In [ ]:
def plot_dark_with_distributions(image, rn, dark_rate, n_images=1,
                                 show_poisson=True, show_gaussian=True):
    h = plt.hist(image.flatten(), bins=20, align='mid', 
                 density=True, label="Dark current counts");
    bins = h[1]
    expected_mean_dark = dark_rate * exposure / gain
    pois = stats.poisson(expected_mean_dark * n_images)
    pois_x = np.arange(0, 300, 1)

    new_area = np.sum(1/n_images * pois.pmf(pois_x))

    if show_poisson:
        plt.plot(pois_x / n_images, pois.pmf(pois_x) / new_area, 
                 label="Poisson dsitribution, mean of {:5.2f} counts".format(expected_mean_dark)) 
    plt.xlim(-20, 30)

    if show_gaussian:
        gauss = stats.norm(loc=expected_mean_dark, scale=rn / gain)
        gauss_x = np.linspace(*plt.xlim(), num=10000)
        plt.plot(gauss_x, gauss.pdf(gauss_x), label='Gaussian, standdard dev is read noise in counts') 
        
    plt.xlabel("Dark counts in {} sec exposure".format(exposure))
    plt.ylabel("Fraction of pixels (area normalized to 1)")
    plt.legend()

In [ ]:
plt.figure(figsize=(20, 10))


plot_dark_with_distributions(darks[-1], high_read_noise, dark_rate, n_images=1)

plt.ylim(0, 0.8)

plt.grid()

#### This dark frame measures noise, not dark current 

The pixel distribution is clearly a Gaussian distribution with a width determined by the read noise, not the underlying Poisson distribution that a dark frame is trying to measure. The only way around this (assuming the dark current is large enough that it needs to be subtracted at all) is to make the exposure long enough that the expected counts exceed the dark current.

We explore that case below by adding in a much smaller amount of noise.

In [ ]:
plt.figure(figsize=(20, 10))

plot_dark_with_distributions(image_average, high_read_noise, dark_rate, n_images=n_images)

plt.ylim(0, 0.8)
plt.grid()

# OOF! WHY IS THIS NOT A POISSON DISTRIBUTION?

## Maybe the average of a bunch of Poisson distributions is not a Poisson distribution and is instead a Gaussian?

Nope, not a gaussian either, but not a Poisson. Note below that the *sum* is a Poisson with mean value `n_images` times larger than the single-image value. To scale to the average, calculate the Poisson distribution with mean value $N_{images} d_C(t)$, plot that as a function of `counts/n_images`, and normalize the resulting distribution.

##### Also, it *is* the expected distribution for a sum of Poissons IF the read noise is zero or small.

### Plot below shows properly calculated Poisson and Gaussian distributions for sum of each type.

In [ ]:
plt.figure(figsize=(20, 10))


h = plt.hist((n_images * image_average).flatten(), bins=20, align='mid', density=True, label="Histogram of dark current counts");
bins = h[1]
expected_mean_dark = dark_rate * exposure / gain
pois = stats.poisson(expected_mean_dark * n_images)
pois_x = np.arange(0, 300, 1)

plt.plot(pois_x, pois.pmf(pois_x), 
         label="Poisson dsitribution, mean of {:5.2f} counts".format(dark_rate * exposure / gain)) 


gauss = stats.norm(loc=expected_mean_dark * n_images, scale=high_read_noise / gain * np.sqrt(n_images))
gauss_x = np.linspace(*plt.xlim())


plt.plot(gauss_x, gauss.pdf(gauss_x), label='Gaussian, standdard dev is read noise in counts') 
plt.xlabel("Dark counts in {} exposure".format(exposure))
plt.ylabel("Number of pixels (area normalized to 1)")
plt.legend()
plt.grid()

### Read noise much lower than dark current 

In this case the read noise is 1 $e^-$, lower than the expected dark current for this exposure time, 10$e^-$.

In [ ]:
low_read_noise = 1

In [ ]:
pixels = np.random.randint(50, high=190, size=n_pixels)
pixel_values = np.zeros(n_images)
pixel_averages = np.zeros(n_images)
base_image = np.zeros([image_size, image_size])
darks = np.zeros([n_images, image_size, image_size])

plt.figure(figsize=(20, 10))
for n in range(n_images):
    darks[n] = dark_current(base_image, dark_rate, exposure, gain=gain, hot_pixels=False)
    darks[n] = darks[n] + read_noise(base_image, low_read_noise, gain=gain)
for pixel in pixels:
    for n in range(n_images):
        pixel_values[n] = darks[n, pixel, pixel]
    plt.plot(pixel_values, label='pixel [{0}, {0}]'.format(pixel), alpha=0.5)
    pixel_averages += pixel_values

image_average = darks.mean(axis=0)

plt.plot(pixel_averages / n_pixels, 
         linewidth=3,
         label='Average over {} pixels'.format(n_pixels))
# plt.xlim(0, n_images - 1)
plt.hlines(dark_rate * exposure / gain, *plt.xlim(), 
           linewidth=3, 
           label="Expected counts")
plt.xlabel('Image number')
plt.ylabel('Counts due to dark current')

plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize=(20, 10))


plot_dark_with_distributions(darks[-1], low_read_noise, dark_rate, n_images=1)

plt.ylim(0, 0.8)

plt.grid()